## CoNLL-2003 Example for Text Extensions for Pandas
### Part 3

To run this notebook, you will need to obtain a copy of the CoNLL-2003 data set's corpus.
Drop the corpus's files into the following locations:
* conll_03/eng.testa
* conll_03/eng.testb
* conll_03/eng.train

If you are unfamiliar with the basics of Text Extensions for Pandas, we recommend you 
start with Part 1 of this example.

### Introduction

At the end of part 2 of the demo, we've shown that there are incorrect labels hidden in the CoNLL-2003 validation set, and that you can pinpoint those incorrect labels by data-mining the results of the 16 models the competitors submitted.

Our goal for part 3 of the demo is to pinpoint incorrect labels across the entire data set. The (rough) process to do so will be:

1. Retokenize the entire corpus using a "BERT-compatible" tokenizer, and map the token/entity labels from the original corpus on to the new tokenization.
2. Generate BERT embeddings for every token in the entire corpus in one pass, and store those embeddings in a dataframe column (of type TensorType) alongside the tokens and labels.
3. Use the embeddings to quickly train multiple models at multiple levels of sophistication (something like: SVMs, random forests, and LSTMs with small and large numbers of hidden states). Split the corpus into 10 parts and perform a 10-fold cross-validation.
4. Repeat the process from part 2 on each fold of the 10-fold cross-validation, comparing the outputs of every model on the validation set for each fold.
5. ?
6. Profit!


In [ ]:
# INITIALIZATION BOILERPLATE

# The Jupyter kernel for this notebook usually starts up inside the notebooks
# directory, but the text_extensions_for_pandas package code is in the parent
# directory. Add that parent directory to the front of the Python include path.
import sys
if (sys.path[0] != ".."):
    sys.path[0] = ".."
    
# Libraries
import numpy as np
import pandas as pd
import torch
import transformers
from typing import *

import matplotlib.pyplot as plt
import ipywidgets
from IPython.display import display

# And of course we need the text_extensions_for_pandas library itself.
import text_extensions_for_pandas as tp

In [ ]:
# Read in the corpus in its original tokenization
gold_standard = tp.conll_2003_to_dataframes("../conll_03/eng.testa")
gold_standard_spans = [tp.iob_to_spans(df) for df in gold_standard]

In [ ]:
gold_standard[0]

In [ ]:
gold_standard_spans[0].head()

In [ ]:
# Retokenize the corpus's text with the BERT tokenizer
bert_model_name = "bert-base-uncased"
tokenizer = transformers.BertTokenizerFast.from_pretrained(bert_model_name, 
                                                           add_special_tokens=True)
bert_toks = [tp.make_bert_tokens(df["char_span"].values[0].target_text, tokenizer) for 
             df in gold_standard]
bert_toks[0]

In [ ]:
# BERT tokenization includes special zero-length tokens.
doc_toks = bert_toks[0]
doc_toks[doc_toks["special_tokens_mask"]]

In [ ]:
# Align the BERT tokens with the original tokenization
# Start by converting the elements of gold_standard_spans to character spans
new_gold_standard_spans = []
for i in range(len(gold_standard_spans)):
    g = gold_standard_spans[i]
    original_spans = g["token_span"]
    bert_token_spans = tp.TokenSpanArray.align_to_tokens(bert_toks[i]["char_span"],
                                                         original_spans)
    new_gold_standard_spans.append(pd.DataFrame({
        "original_span": original_spans,
        "bert_token_span": bert_token_spans,
        "ent_type": g["ent_type"]
    }))                                            


In [ ]:
new_gold_standard_spans[1].head()

In [ ]:
# Generate IOB2 tags that align with the BERT tokens.
# See https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)
bert_gold_standard = [t.copy() for t in bert_toks]
for i in range(len(bert_gold_standard)):
    bert_gold_standard[i]["ent_iob"] = (
        tp.spans_to_iob(new_gold_standard_spans[i]["bert_token_span"]))

In [ ]:
# Double-check that the new IOB2 tags are properly aligned
bert_gold_standard[1].head(20)

In [ ]:
# Add the entity type tags to the new IOB2 data
for i in range(len(bert_gold_standard)):
    iob_df = bert_gold_standard[i]
    entity_df = new_gold_standard_spans[i]
    tmp_df = (
        tp
        .contain_join(entity_df["bert_token_span"], iob_df["token_span"],
                      "bert_token_span", "token_span")
        .merge(entity_df[["bert_token_span", "ent_type"]])
        [["token_span", "ent_type"]]
    )
    bert_gold_standard[i] = iob_df.merge(tmp_df, how="left")
    
# Print out the dataframe from the previous cell to show it has a new column
bert_gold_standard[1].head(20)

In [ ]:
# Now we need to attach some embeddings. Fire up a canned BERT model.
bert = transformers.BertModel.from_pretrained(bert_model_name)

In [ ]:
# Generate embeddings for document 1, to show how it's done
toks_df = bert_gold_standard[1]
input_ids = torch.from_numpy(toks_df["input_id"].values.reshape([1,-1]))
bert_result = bert(input_ids=input_ids)
hidden_states = bert_result[0]
hidden_states.detach().numpy()

In [ ]:
# Code to split documents into overlapping windows for computing embeddings
# and to merge the results back.
def seq_to_windows(seq: np.ndarray, overlap: int, non_overlap: int) \
        -> Dict[str, np.ndarray]:
    """
    Convert a variable-length sequence into a set of fixed length windows,
    adding padding as necessary.
    
    :param seq: Original variable length sequence, as a 1D numpy array
    :param overlap: How much overlap there should be between adjacent windows
    :param non_overlap: How much non-overlapping content between the overlapping
     regions there should be at the middle of each window?
     
    :returns: A dictionary with two entries:
      * `input_ids`: 2D `np.ndarray` of fixed-length windows
      * `attention_masks`: 2D `np.ndarray` of attention masks (1 for
        tokens that are NOT masked, 0 for tokens that are masked)
        to feed into your favorite BERT-like embedding generator.
    """
    if len(seq.shape) != 1:
        raise ValueError(f"Input array must be 1D; got shape {seq.shape}")
    window_length, pre_padding, post_padding = _compute_padding(
        len(seq), overlap, non_overlap)
    
    # First generate the windows as a padded flat arrays.
    padded_length = len(seq) + pre_padding + post_padding
    buf = np.zeros(shape=[padded_length], dtype=seq.dtype)
    buf[pre_padding:pre_padding + len(seq)] = seq
    
    mask_buf = np.zeros_like(buf, dtype=int)  # 0 == MASKED
    mask_buf[pre_padding:pre_padding + len(seq)] = 1  # 1 == NOT MASKED

    # Reshape the flat arrays into overlapping windows.
    num_windows = padded_length // (overlap + non_overlap)
    windows = np.zeros(shape=[num_windows, window_length], dtype=seq.dtype)
    masks = np.zeros(shape=[num_windows, window_length], dtype=int)
    for i in range(num_windows):
        start = i * (overlap + non_overlap)
        windows[i] = buf[start:start+window_length]
        masks[i] = mask_buf[start:start+window_length]
    return {
        "input_ids": windows,
        "attention_masks": masks
    }

def windows_to_seq(seq: np.ndarray, windows: np.ndarray,
                   overlap: int, non_overlap: int) \
        -> Dict[str, np.ndarray]:
    """
    Inverse of `seq_to_windows()`
    Convert fixed length windows with padding to a variable-length sequence
    that matches up with the original sequence from which the windows were
    computed.
    
    :param seq: Original variable length sequence to align with,
      as a 1D numpy array
    :param windows: Windowed data to align with the original sequence.
      Usually this data is the result of applying a transformation to the
      output of `seq_to_windows()`
    :param overlap: How much overlap there is between adjacent windows
    :param non_overlap: How much non-overlapping content between the overlapping
     regions there should be at the middle of each window?
     
    :returns: A 1D `np.ndarray` containing the contents of `windows` that 
     correspond to the elements of `seq`.
    """
    if len(seq.shape) != 1:
        raise ValueError(f"Input array must be 1D; got shape {seq.shape}")
    window_length, pre_padding, post_padding = _compute_padding(
        len(seq), overlap, non_overlap)
    
    # Input may be an array of n-dimensional tensors.
    result_shape = [len(seq)] + list(windows.shape[2:])
    #result = np.zeros_like(seq, dtype=windows.dtype)
    result = np.zeros(shape=result_shape, dtype=windows.dtype)
    
    # Special-case the first and last windows.
    if len(seq) <= non_overlap + (overlap//2):
        # Only one window, potentially a partial window.
        return windows[0][overlap:overlap+len(seq)]
    else:
        result[0:non_overlap+(overlap//2)] = \
            windows[0][overlap:overlap+non_overlap+(overlap//2)]
    
    num_to_copy_from_last = overlap//2 + overlap + non_overlap - post_padding
    if num_to_copy_from_last > 0:
        result[-num_to_copy_from_last:] = \
            windows[-1][overlap//2:(overlap//2)+num_to_copy_from_last]
        
    # Remaining windows can be covered in a loop
    for i in range(1, len(windows) - 1):
        src_start = overlap//2
        dest_start = (overlap//2 + non_overlap
                      + (i - 1) * (overlap + non_overlap))
        num_to_copy = min(non_overlap + overlap, len(seq) - dest_start)
        result[dest_start:dest_start+num_to_copy] = \
            windows[i][src_start:src_start+num_to_copy]
            
    return result
        
def _compute_padding(seq_len, overlap, non_overlap) \
        -> Tuple[int, int, int, int]:
    """
    Shared padding computation for seq_to_windows() and windows_to_seq()
    
    :param seq_len: Length of original sequence
    :param overlap: How much overlap there should be between adjacent window
    :param non_overlap: How much non-overlapping content between the overlapping
     regions there should be at the middle of each window?
     
    :returns: A tuple of (window_length, pre_padding, post_padding)
    """
    if 0 != overlap % 2:
        raise ValueError(f"Non-even overlaps not implemented; got {overlap}")
        
    # Each window has overlapping regions at the beginning and end
    window_length = (2 * overlap) + non_overlap

    # Account for the padding before the first window
    pre_padding = overlap
    
    # Account for the padding after the last window
    remainder = (seq_len + pre_padding) % (overlap + non_overlap)
    post_padding = window_length - remainder
    if post_padding == window_length:
        # Chop off empty last window
        post_padding -= overlap + non_overlap
    
    return (window_length, pre_padding, post_padding)
    

In [ ]:
# Test the functions we just wrote.
for seqlen in range(1, 20):
    seq = np.arange(1, seqlen)
    seq_after = windows_to_seq(seq, seq_to_windows(seq, 2, 3)["input_ids"], 2, 3)
    if np.any(seq != seq_after):
        raise ValueError("Before: {seq}; After: {seq_after}")
        
for seqlen in range(200, 400):
    # print(f"seqlen: {seqlen}")
    seq = np.arange(1, seqlen)
    windows = seq_to_windows(seq, 32, 64)
    # print(f"input_ids: {windows['input_ids']}")
    seq_after = windows_to_seq(seq, windows["input_ids"], 32, 64)
    if np.any(seq != seq_after):
        raise ValueError("Before: {seq}; After: {seq_after}")
        
for seqlen in range(50, 100):
    # print(f"seqlen: {seqlen}")
    seq = np.arange(1, seqlen)
    windows = seq_to_windows(seq, 32, 64)
    # print(f"input_ids: {windows['input_ids']}")
    seq_after = windows_to_seq(seq, windows["input_ids"], 32, 64)
    if np.any(seq != seq_after):
        raise ValueError("Before: {seq}; After: {seq_after}")

In [ ]:
# Run document 1 in batches of 128 with the first and last 32 in each
# batch overlapping with adjacent windows
_OVERLAP = 32
_NON_OVERLAP = 64

toks_df = bert_gold_standard[1]
flat_input_ids = toks_df["input_id"].values
windows = seq_to_windows(flat_input_ids, _OVERLAP, _NON_OVERLAP)
bert_result = bert(input_ids=torch.tensor(windows["input_ids"]), 
                   attention_mask=torch.tensor(windows["attention_masks"]))
hidden_states = windows_to_seq(flat_input_ids, 
                               bert_result[0].detach().numpy(),
                               _OVERLAP, _NON_OVERLAP)
hidden_states

In [ ]:
# Define a function that adds embeddings to a dataframe.
def add_embeddings(df: pd.DataFrame) -> pd.DataFrame:
    _OVERLAP = 32
    _NON_OVERLAP = 64
    flat_input_ids = df["input_id"].values
    windows = seq_to_windows(flat_input_ids, _OVERLAP, _NON_OVERLAP)
    bert_result = bert(input_ids=torch.tensor(windows["input_ids"]), 
                       attention_mask=torch.tensor(windows["attention_masks"]))
    hidden_states = windows_to_seq(flat_input_ids, 
                                   bert_result[0].detach().numpy(),
                                   _OVERLAP, _NON_OVERLAP)
    embeddings = tp.TensorArray(hidden_states)
    ret = df.copy()
    ret["embeddings"] = embeddings
    return ret
    
add_embeddings(bert_gold_standard[2])

In [ ]:
# Now we can add embeddings to the entire gold standard data set.
# This takes a while, so we display a progress bar.
num_docs = len(bert_gold_standard)
progress_bar = ipywidgets.IntProgress(0, 0, num_docs,
                                      description="Running...",
                                      layout=ipywidgets.Layout(width='100%'))
display(progress_bar)
for i in range(len(bert_gold_standard)):
    bert_gold_standard[i] = add_embeddings(bert_gold_standard[i])
    progress_bar.value = i + 1
    progress_bar.description = f"{i + 1}/{num_docs} docs"
progress_bar.bar_style = "success"

In [ ]:
# What does the dataframe for document 5 look like now?
bert_gold_standard[5]